# 1 - Récupération des données

Les réglementations étant assez vastes, nous allons dans un premier temps nous concentrer sur les réglementations liées à la sécurité incendie.

Cette dernière est propre aux types d'ouvrages construits parmi lesquels nous retrouvons :
- les ERT ou établissement recevant des travailleurs
- les ERP ou établissement recevant du publics
- les batiments d'habitation
(etc) et tous ont leurs propres lois et décrets.

## 1-1 Analyse des données du Code du travail (ERT)

### 1-1-1 Scrapping de Legifrance

In [45]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.legifrance.gouv.fr/codes/section_lc/LEGITEXT000006072050/LEGISCTA000018488606/#LEGISCTA000018532586"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    data = []
    titre = None  
    chapitre = None  
    section = None 

    for span in soup.find_all('span', {'data-anchor': True}):
        text = span.get_text(strip=True)

        if "Titre" in text:  
            titre = text
            chapitre = None 
            csection = None
        elif "Chapitre" in text and titre: 
            chapitre = text
            section = None  
        elif "Section" in text and titre and chapitre:  
            section = text

            articles = span.find_parent('li').find_all('article', class_="js-child list-article-consommation")
            
            for article in articles:
                article_name_tag = article.find('a', href=lambda x: x and 'codes/article_lc' in x)
                article_name = article_name_tag.get_text(strip=True) if article_name_tag else 'Article non disponible'

                decree = article.find('a', href=lambda x: x and '/loda/id/' in x)
                decree_text = decree.get_text(strip=True) if decree else 'Décret non disponible'

                content_div = article.find('div', class_='content')
                content = content_div.get_text(separator="\n", strip=True) if content_div else 'Contenu non disponible'

                data.append({
                    'Titre': titre,
                    'Chapitre': chapitre,
                    'Section': section,
                    'Article': article_name,
                    'Décret': decree_text,
                    'Contenu': content
                })

    df_final = pd.DataFrame(data)
    display(df_final)

else:
    print("Erreur lors du chargement :", response.status_code)


,Titre,Chapitre,Section,Article,Décret,Contenu
0,Titre Ier : Obligations du maître d'ouvrage po...,Chapitre Ier : Principes généraux (Articles R4...,Section 1 : Champ d'application et définitions...,Article R4211-1,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Les dispositions du présent titre déterminent,..."
1,Titre Ier : Obligations du maître d'ouvrage po...,Chapitre Ier : Principes généraux (Articles R4...,Section 1 : Champ d'application et définitions...,Article R4211-2,Création Décret n°2008-244 du 7 mars 2008 - ar...,"Pour l'application du présent titre, on entend..."
2,Titre Ier : Obligations du maître d'ouvrage po...,Chapitre Ier : Principes généraux (Articles R4...,Section 2 : Dossier de maintenance (Articles R...,Article R4211-3,Modifié par Décret n°2011-1461\n du 7 novembre...,Le maître d'ouvrage élabore et transmet aux ut...
3,Titre Ier : Obligations du maître d'ouvrage po...,Chapitre Ier : Principes généraux (Articles R4...,Section 2 : Dossier de maintenance (Articles R...,Article R4211-4,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...
4,Titre Ier : Obligations du maître d'ouvrage po...,Chapitre Ier : Principes généraux (Articles R4...,Section 2 : Dossier de maintenance (Articles R...,Article R4211-5,Création Décret n°2008-244 du 7 mars 2008 - ar...,Le dossier de maintenance des lieux de travail...
...,...,...,...,...,...,...
281,Titre II : Obligations de l'employeur pour l'u...,"Chapitre VIII : Installations sanitaires, res...",Section 3 : Hébergement (Articles R4228-26 à R...,Article R4228-33,Création Décret n°2008-244 du 7 mars 2008 - ar...,Des lavabos à eau potable et à température rég...
282,Titre II : Obligations de l'employeur pour l'u...,"Chapitre VIII : Installations sanitaires, res...",Section 3 : Hébergement (Articles R4228-26 à R...,Article R4228-34,Création Décret n°2008-244 du 7 mars 2008 - ar...,Des cabinets d'aisance et des urinoirs sont in...
283,Titre II : Obligations de l'employeur pour l'u...,"Chapitre VIII : Installations sanitaires, res...",Section 3 : Hébergement (Articles R4228-26 à R...,Article R4228-35,Création Décret n°2008-244 du 7 mars 2008 - ar...,Des douches à température réglable sont instal...
284,Titre II : Obligations de l'employeur pour l'u...,"Chapitre VIII : Installations sanitaires, res...",Section 3 : Hébergement (Articles R4228-26 à R...,Article R4228-36,Modifié par Ordonnance n°2010-462 du 6 mai 201...,Les dispositions des articles\nR. 4228-26 à R....


### 1-1-2 Pre-processing des données

## 1-2 Analyse des données des Etablissements recevant du public (ERP)

## 1-3 Analyse des données du Code de la construction et de l’habitation (CCH)